# Fig 1 just functions

In [10]:
# Import my packages
import matplotlib.pyplot as plt
import numpy as np
import math
from mpl_toolkits import mplot3d
from matplotlib import animation
from scipy.fft import fft, ifft, ifft2, fftfreq, fft2, fftshift
from scipy import signal

In [11]:
def gauss_FT(FWHM, x_range): # y_range = 0 1D, y_range = x_range 2D
    x = x_range
    sigma = FWHM*(1/(2*(2*np.log(2))**(1/2))) 
    h = (1 / (2 * np.pi * (sigma ** 2)) ** (1 / 2)) * np.exp(-(x ** 2) / (2 * sigma ** 2))
    a = fft(h)
    sample_freq_x = fftfreq(len(x), d=0.5)
    h_shifted = fftshift(a)
    sample_freq_x = fftshift(sample_freq_x)
    return(h,h_shifted,sample_freq_x)

In [12]:
##### temporal filters and their FTs
def time_filter(tau,t, time_step):
    theta = 1 # Define Heaviside for t-t'>0 , t'=0
    f = 2*tau**(-3/2)*t*theta*np.exp(-t/tau)
    f_dev = 2*tau**(-3/2)*(tau-t)*theta*np.exp(-t/tau)
    a=fft(f)
    amplitude = np.abs(a)
    n= len(t)
    sample_freq = fftfreq(n, time_step)
    # Shift the Fourier transform to center the zero-frequency component
    f_shifted = fftshift(a)
    sample_freq_shifted = fftshift(sample_freq)
    return(sample_freq_shifted,f_shifted)
def time_filter_der(tau, t, time_step):
    theta = 1 # Define Heaviside for t-t'>0 , t'=0
    f = 2*tau**(-3/2)*t*theta*np.exp(-t/tau)
    f_dev = 2*tau**(-3/2)*(tau-t)*theta*np.exp(-t/tau)
    a=fft(f_dev)
    amplitude = np.abs(a)
    n= len(t)
    sample_freq = fftfreq(n, time_step)
    # Shift the Fourier transform to center the zero-frequency component
    f_dev_shifted = fftshift(a)
    sample_freq_shifted_dev = fftshift(sample_freq)
    return(sample_freq_shifted_dev,f_dev_shifted)

In [13]:
def edge_stimulus(which_stimulus, t_range, x_range, x_initial): #FTB_ON,FTB_OFF,BTF_OFF,BTF_ON #t = np.arange(0, 8, 1/240) # x = np.arange(-180, 180, 0.5)
    if which_stimulus == 'FTB_ON':
        dx = -(x_range[:, np.newaxis] - 30 * t_range - x_initial)
        c = np.heaviside(dx, 0)
    elif which_stimulus == 'FTB_OFF':
        dx = (x_range[:, np.newaxis] - 30 * t_range - x_initial)
        c = np.heaviside(dx, 0)
    elif which_stimulus == 'BTF_OFF':
        dx = (-x_range[:, np.newaxis] - 30 * t_range - x_initial)
        c = np.heaviside(dx, 0)
    elif which_stimulus == 'BTF_ON':
        dx = -(-x_range[:, np.newaxis] - 30 * t_range - x_initial)
        c = np.heaviside(dx, 0)
    return(c)

In [14]:
def gratings(c_o, f, k, t_range, x_range):
    omega=2*math.pi*f
    t=t_range 
    x=x_range # x in degrees, I just need x to be larger than  t
    contrast = np.empty([len(x),len(t)]) # now instead of creating empty arrays with contrast = np.array([]) and fill with append, I declare the size & it's faster
    for j, x_value in enumerate(x):
        contrast[j,:] = c_o*np.sin(omega*t-k*x_value)   # c= c_o*np.sin(omega*t[i]-k*x[k]) 
    c = contrast
    return(c)

In [15]:
def contrast_fourier(c,x_range,t_range):
    # contrast signal Fourier space
    t = t_range
    x = x_range
    a=fft2(c)
    c_shifted = fftshift(a)
    #amplitude = np.abs(a)
    # Calculate the corresponding spatial and temporal frequencies
    spatial_freq = fftfreq(len(x), d=x[1] - x[0])
    temporal_freq = fftfreq(len(t), d=t[1] - t[0])

    # Shift the spatial and temporal frequencies to be centered??
    spatial_freq_shifted = fftshift(spatial_freq) #don't need it because already have it in : sample_freq_x & sample_freq_shifted_dev
    temporal_freq_shifted = fftshift(temporal_freq)

    # Plot the magnitude of the 2D Fourier transform
    #plt.imshow(np.log(np.abs(c_shifted)), extent=[spatial_freq_shifted[0], spatial_freq_shifted[-1], temporal_freq_shifted[0], temporal_freq_shifted[-1]], aspect ='auto')
    #plt.colorbar()
    #plt.title('Fourier Transform of contrast')
    #plt.ylabel('time freq') # CHECK AGAIN MARIA
    #plt.xlabel('spatial freq')
    return(c_shifted)

In [16]:
#### for the f
def filtered_contrast_f(sample_freq_x, sample_freq_shifted,c_shifted, f_shifted,h_shifted):
    filtered_c_f = np.empty([len(sample_freq_x),len(sample_freq_shifted)],dtype = 'complex_')
    for i, fx_value in enumerate(sample_freq_x):# fx
        filtered_c_f[i,:] = c_shifted[i,:]*f_shifted[:]*h_shifted[i] #because f_shifted, h_shifted is 1D
    # now invert the FT
    filtered_c_f = fftshift(filtered_c_f)
    filtered_c_f = ifft2(filtered_c_f)
    filtered_c_f = filtered_c_f.real
    return(filtered_c_f)

def filtered_contrast_f_dev(sample_freq_x, sample_freq_shifted_dev, c_shifted,f_dev_shifted,h_shifted):
    filtered_c_dev = np.empty([len(sample_freq_x),len(sample_freq_shifted_dev)],dtype = 'complex_')
    for i, fx_value in enumerate(sample_freq_x):# fx
        filtered_c_dev[i,:] = c_shifted[i,:]*f_dev_shifted[:]*h_shifted[i] 
    # now invert the FT
    filtered_c_dev = fftshift(filtered_c_dev)
    filtered_c_dev = ifft2(filtered_c_dev)
    filtered_c_dev = filtered_c_dev.real
    return(filtered_c_dev)

In [17]:
def ramp(y):
    return np.maximum(0, y)

In [18]:
# c=np.sin()  #g_l=1/10 #g_inh = 0.3*g_l #g_l=1/10 #g_exc = 0.1*g_l
def g_vm_cal_calculation(filtered_c,filtered_c_dev):
    def ramp(y):
        return np.maximum(0, y) # ramp function is contrast dependent
    E_exc = 60 # mV
    E_inh = -30 # mV
    x=np.arange(-180, 180, 0.5) # x=np.arange(-180, 180, 0.5) 
    V_m = np.empty([len(x),len(t)])
    calcium = np.empty([len(x),len(t)])
    g1_div_g_l = np.empty([len(x),len(t)])
    g2_div_g_l = np.empty([len(x),len(t)])
    g3_div_g_l = np.empty([len(x),len(t)])
    for i,x_value in enumerate(x[:720]):  #s should be an array calculated over x so the index changes by x (was 710 but works for 720 or len(x))

        g1_index = (i - 10) % 720 # Here I make sure that the peripheral neurons get input from spatially seperated points in space 
        g3_index = (i + 10) % 720 # (index circles around so if i=719 then g3 gets input from i=9 beacuse of this mod )

        g1_div_g_l[i, :] = 0.3 * (ramp(-filtered_c[g1_index, :]))
        g2_div_g_l[i, :] = 0.1 * (ramp(filtered_c_dev[i, :]))
        g3_div_g_l[i, :] = 0.3 * (ramp(filtered_c[g3_index, :]))

        # actually i cannot compute the g1,2,3 seperatly because the membrane potential is always g1/g_L
        ##g1_div_g_l[i,:] =  0.3*(ramp(-filtered_c[i-10,:]))          #Mi9 g1/g_l   Δx is 5 deg
        ##g2_div_g_l[i,:] =  0.1*(ramp(filtered_c_dev[i,:]))          #Tm3 g2/g_l
        ##g3_div_g_l[i,:] =  0.3*(ramp(filtered_c[i+10,:]))           #Mi4 g3/g_l
        #V_m[i,j] = (g1_div_g_l[i,j]*E_inh+g2_div_g_l[i,j]*E_exc+g3_div_g_l[i,j]*E_inh)/(1+0.3+0.1+0.3) # incorporate my LIF code but first make sure the g are right
        
        ##### Calculate Membrane Potential
        V_m[i,:] = (g1_div_g_l[i,:]*E_inh + g2_div_g_l[i,:]*E_exc + g3_div_g_l[i,:]*E_inh)/(1+g1_div_g_l[i,:]+g2_div_g_l[i,:]+g3_div_g_l[i,:])
        calcium[i,:] = (ramp(V_m[i,:]))**2
    return(g1_div_g_l,g2_div_g_l,g3_div_g_l,V_m,calcium)